In [4]:
import re
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as ss
import matplotlib.pyplot as plt
from matplotlib import style, colors
from matplotlib import gridspec
import random
import string
import math

# matplotlib.style.use('ggplot')
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12, 6)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [86]:
def filter_snv(vcf, qss):
    df = pd.read_csv(vcf, sep='\t', comment='#', header=None)
    df.columns = ["#CHROM",  "POS",  "ID",  "REF",  "ALT",  "QUAL",  "FILTER",  "INFO",  "FORMAT",  "NORMAL",  "TUMOR"]
    df["QSS"] = df['INFO'].apply(lambda x: [i.split('=')[1] for i in x.split(';') if i.startswith('QSS')][0])
    print(df.shape)
    df = df[df.QSS.astype(int)>=qss].drop('QSS', axis=1)
    print(df.shape)
    return df

In [76]:
snv_vcf = '/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-03-06-02008/hg19a/GSH/A37241_A37166/strelka/16522/bwa/results/passed.somatic.snvs.eff.dbSNP_v137.cosmic_v64.annotations.vcf'
df_snv = filter_snv(snv_vcf, 30)
df_snv.head(2)

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,NORMAL,TUMOR
10,1,2903222,.,C,T,.,PASS,NT=ref;QSS=61;QSS_NT=61;SGT=CC->CT;SOMATIC;TQS...,DP:FDP:SDP:SUBDP:AU:CU:GU:TU,"37:0:0:0:0,0:37,37:0,0:0,0","63:1:0:0:0,0:38,39:0,0:24,25"
11,1,3282979,.,C,T,.,PASS,NT=ref;QSS=30;QSS_NT=30;SGT=CC->CT;SOMATIC;TQS...,DP:FDP:SDP:SUBDP:AU:CU:GU:TU,"23:0:0:0:0,0:23,23:0,0:0,0","60:0:0:0:0,0:33,33:0,0:27,27"


In [87]:
def filter_indel(vcf, qsi):
    df = pd.read_csv(vcf, sep='\t', comment='#', header=None)
    df.columns = ["#CHROM",  "POS",  "ID",  "REF",  "ALT",  "QUAL",  "FILTER",  "INFO",  "FORMAT",  "NORMAL",  "TUMOR"]
#     print(df.head(2))
    df["QSI"] = df['INFO'].apply(lambda x: [i.split('=')[1] for i in x.split(';') if i.startswith('QSI')][0])
    print(df.shape)
    df = df[df.QSI.astype(int)>=qsi].drop('QSI', axis=1)
    print(df.shape)
    return df

In [78]:
indel_vcf = '/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-03-06-02001/hg19a/GSH/A37234_A37159/strelka/16551/bwa/results/passed.somatic.indels.eff.dbSNP_v137.cosmic_v64.annotations.vcf'
df_indel = filter_indel(indel_vcf, 30)
df_indel.head(2)

  CHROM      POS ID REF        ALT QUAL FILTER  \
0     1  5220251  .   A  ATATATACT    .   PASS   
1     1  5220284  .   C    CACACTA    .   PASS   

                                                INFO  \
0  IC=1;IHP=2;NT=ref;QSI=36;QSI_NT=36;RC=0;RU=TAT...   
1  IC=1;IHP=2;NT=ref;QSI=55;QSI_NT=55;RC=0;RU=ACA...   

                                  FORMAT  \
0  DP:DP2:TAR:TIR:TOR:DP50:FDP50:SUBDP50   
1  DP:DP2:TAR:TIR:TOR:DP50:FDP50:SUBDP50   

                                       NORMAL  \
0        76:76:72,100:0,0:5,3:82.15:9.86:0.00   
1  124:124:114,164:0,0:10,3:111.99:16.62:0.00   

                                         TUMOR  
0  181:181:140,185:4,4:38,44:197.34:17.97:0.00  
1  303:303:251,354:9,15:40,37:265.8:37.47:0.00  
(1242, 12)
(1242, 11)


,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,NORMAL,TUMOR
0,1,5220251,.,A,ATATATACT,.,PASS,IC=1;IHP=2;NT=ref;QSI=36;QSI_NT=36;RC=0;RU=TAT...,DP:DP2:TAR:TIR:TOR:DP50:FDP50:SUBDP50,"76:76:72,100:0,0:5,3:82.15:9.86:0.00","181:181:140,185:4,4:38,44:197.34:17.97:0.00"
1,1,5220284,.,C,CACACTA,.,PASS,IC=1;IHP=2;NT=ref;QSI=55;QSI_NT=55;RC=0;RU=ACA...,DP:DP2:TAR:TIR:TOR:DP50:FDP50:SUBDP50,"124:124:114,164:0,0:10,3:111.99:16.62:0.00","303:303:251,354:9,15:40,37:265.8:37.47:0.00"


In [80]:
outdir = '/projects/trans_scratch/validations/workspace/szong/Cervical/mutsig2cv/filtered_vcfs/'
df_snv.to_csv('{}a.snvs.vcf'.format(outdir), index=False, sep='\t')
df_indel.to_csv('{}a.indels.vcf'.format(outdir), index=False, sep='\t')

In [88]:
vcfs = '/projects/trans_scratch/validations/workspace/szong/Cervical/mutsig2cv/filtered_vcfs/118_strelka_vcfs.txt'
df = pd.read_csv(vcfs, sep='\t', header=None)
df.columns = ['patient', 'snv_vcf', 'indel_vcf']
df.head(2)

,patient,snv_vcf,indel_vcf
0,HTMCP-03-06-02001,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...
1,HTMCP-03-06-02002,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...


In [91]:
for _, row in df.iterrows():
    patient = row['patient']
    snv_vcf = row['snv_vcf']
    print('processing {}'.format(patient))
#     print(snv_vcf, indel_vcf)
    df_snv = filter_snv(snv_vcf, 30)
    indel_vcf = row['indel_vcf']
#     print('processing {}, indel'.format(patient))
    df_indel = filter_indel(indel_vcf, 35)
    f_name = '{0}{1}.snvs.vcf'.format(outdir, patient)
    with open(f_name, 'a') as f: 
        f.write('##fileformat=VCFv4.1\n')
        df_snv.to_csv(f, index=False, sep='\t')
        
    indel_name = '{0}{1}.indels.vcf'.format(outdir, patient)
    with open(indel_name, 'a') as g: 
        g.write('##fileformat=VCFv4.1\n')
        df_indel.to_csv(g, index=False, sep='\t')
    

processing HTMCP-03-06-02001
(25732, 12)
(22747, 11)
(1242, 12)
(1193, 11)


21

21

processing HTMCP-03-06-02002
(15964, 12)
(12983, 11)
(697, 12)
(632, 11)


21

...

21

21

processing HTMCP-03-06-02447
(13417, 12)
(5402, 11)
(497, 12)
(326, 11)


21

21

In [17]:
df_indel = pd.read_csv(indel_vcf, sep='\t', comment='#', header=None)
df_indel.columns = ["CHROM",  "POS",  "ID",  "REF",  "ALT",  "QUAL",  "FILTER",  "INFO",  "FORMAT",  "NORMAL",  "TUMOR"]
df_indel["QSI"] = df_indel['INFO'].apply(lambda x: x.split(';')[3].split('=')[1])
df_indel.head(2)

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,NORMAL,TUMOR,QSI
0,1,66329,.,ATATATTTTAT,A,.,PASS,IC=0;IHP=6;NT=ref;QSI=34;QSI_NT=34;RC=1;RU=TAT...,DP:DP2:TAR:TIR:TOR:DP50:FDP50:SUBDP50,"82:82:82,111:0,0:5,4:80.67:6.75:0.00","156:156:155,231:6,9:19,28:152.85:23.27:0.00",34
1,1,8573595,.,T,TG,.,PASS,IC=7;IHP=6;NT=ref;QSI=31;QSI_NT=31;RC=6;RU=G;S...,DP:DP2:TAR:TIR:TOR:DP50:FDP50:SUBDP50,"52:52:51,51:0,0:0,0:51.65:0.00:0.00","80:80:59,62:13,13:6,5:78.81:0.00:0.00",31
